In [15]:

# coding: utf-8

# # DECISION TREE IMPLEMENTATION- ENTROPY AND INFO GAIN

# In[4]:


import pandas as pd 
from pandas import DataFrame

df_tennis = pd.read_csv(r"lab2_play_tennis.csv")


# In[6]:

def entropy(probs): 
    import math 
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

def entropy_of_list(a_list): 
    from collections import Counter     
    cnt = Counter(x for x in a_list) 
    print("No and Yes Classes:",a_list.name,cnt)     
    num_instances = len(a_list)*1.0     
    probs = [x / num_instances for x in cnt.values()]     
    return entropy(probs) # Call Entropy: 
total_entropy = entropy_of_list(df_tennis['PlayTennis']) 
print("Entropy of given PlayTennis Data Set:",total_entropy) 


# In[13]:


def information_gain(df, split_attribute_name, target_attribute_name, trace=0): 
    df_split = df.groupby(split_attribute_name) 
    for name,group in df_split:         
        print(name)         
        print(group)    
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name] 
    df_agg_ent.columns = ['Entropy', 'PropObservations'] 
    new_entropy = sum(df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name]) 
    return old_entropy - new_entropy


# In[18]:


print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'Outlook', 'PlayTennis')),"\n") 
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'Humidity', 'PlayTennis')),"\n") 
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'Windy', 'PlayTennis')),"\n")
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis , 'Temperature','PlayTennis')),"\n") 


# In[21]:


def id3(df, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter     
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:         
        return next(iter(cnt)) 
    elif df.empty or (not attribute_names): 
             return default_class 
    else:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] 
        index_of_max = gainz.index(max(gainz)) 
        best_attr = attribute_names[index_of_max] 
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr): 
            subtree = id3(data_subset,
                          target_attribute_name,                         
                          remaining_attribute_names,                         
                          default_class)
            tree[best_attr][attr_val] = subtree 
        return tree
     
        


# In[22]:


attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('PlayTennis') 
print("Predicting Attributes:", attribute_names)


# In[23]:


from pprint import pprint 
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n") 
pprint(tree) 


No and Yes Classes: PlayTennis Counter({'Yes': 9, 'No': 5})
Entropy of given PlayTennis Data Set: 0.9402859586706309
Overcast
    Day   Outlook Temperature Humidity   Windy PlayTennis
2    D3  Overcast         Hot     High    Weak        Yes
6    D7  Overcast        Cool   Normal  Strong        Yes
11  D12  Overcast        Mild     High  Strong        Yes
12  D13  Overcast         Hot   Normal    Weak        Yes
Rain
    Day Outlook Temperature Humidity   Windy PlayTennis
3    D4    Rain        Mild     High    Weak        Yes
4    D5    Rain        Cool   Normal    Weak        Yes
5    D6    Rain        Cool   Normal  Strong         No
9   D10    Rain        Mild   Normal    Weak        Yes
13  D14    Rain        Mild     High  Strong         No
Sunny
    Day Outlook Temperature Humidity   Windy PlayTennis
0    D1   Sunny         Hot     High    Weak         No
1    D2   Sunny         Hot     High  Strong         No
7    D8   Sunny        Mild     High    Weak         No
8    D9   Sun